In [1]:
import json
import numpy as np
import pandas as pd
from urllib.error import URLError, HTTPError
from urllib.request import urlopen

from datetime import datetime
from dateutil.parser import parse

In [2]:
'''
Code copied from https://quantcorner.wordpress.com/2014/11/18/downloading-eias-data-with-python/ on 8/13/2018
'''

class EIAgov(object):
    def __init__(self, token, series):
        '''
        Purpose:
        Initialise the EIAgov class by requesting:
        - EIA token
        - id code(s) of the series to be downloaded

        Parameters:
        - token: string
        - series: string or list of strings
        '''
        self.token = token
        self.series = series

    def Raw(self, ser):
            # Construct url
            url = 'http://api.eia.gov/series/?api_key=' + self.token + '&series_id=' + ser.upper()
            #url = 'http://api.eia.gov/category/?api_key=' + self.token + '&category_id=' + ser.id

            try:
                # URL request, URL opener, read content
                response = urlopen(url);
                raw_byte = response.read()
                raw_string = str(raw_byte, 'utf-8-sig')
                jso = json.loads(raw_string)
                return jso

            except HTTPError as e:
                print('HTTP error type.')
                print('Error code: ', e.code)

            except URLError as e:
                print('URL type error.')
                print('Reason: ', e.reason)

    def GetData(self):
            # Deal with the date series                       
            date_ = self.Raw(self.series[0])        
            date_series = date_['series'][0]['data']
            endi = len(date_series) # or len(date_['series'][0]['data'])
            date = []
            for i in range (endi):
                date.append(date_series[i][0])

            # Create dataframe
            df = pd.DataFrame(data=date)
            df.columns = ['Date']

            # Deal with data
            lenj = len(self.series)
            for j in range (lenj):
                data_ = self.Raw(self.series[j])
                data_series = data_['series'][0]['data']
                data = []
                endk = len(date_series)         
                for k in range (endk):
                    data.append(data_series[k][1])
                df[self.series[j]] = data

            return df



In [3]:
#Download multiple BA demand data
#To do:
#Check that df_all date range are earliest and latest dates for all file
#download all time series data per BA and do quality checks; but first check if EIA will release code they use for QC

if __name__ == '__main__':
    tok = '6d666bb7097e41102ef69a35aa1edb2b'

    demand_list = ['EBA.AVA-ALL.D.H',
                   'EBA.AZPS-ALL.D.H',
                   'EBA.BANC-ALL.D.H',
                   'EBA.BPAT-ALL.D.H',
                   'EBA.CHPD-ALL.D.H',
                   'EBA.CISO-ALL.D.H',
                   'EBA.DOPD-ALL.D.H',
                   'EBA.EPE-ALL.D.H',
                   'EBA.GCPD-ALL.D.H',
                   'EBA.IID-ALL.D.H',
                   'EBA.IPCO-ALL.D.H',
                   'EBA.LDWP-ALL.D.H',
                   'EBA.NEVP-ALL.D.H',
                   'EBA.NWMT-ALL.D.H',  
                   'EBA.PACE-ALL.D.H',
                   'EBA.PACW-ALL.D.H',
                   'EBA.PGE-ALL.D.H',
                   'EBA.PSCO-ALL.D.H',
                   'EBA.PSEI-ALL.D.H',
                   'EBA.SRP-ALL.D.H',
                   'EBA.SCL-ALL.D.H',
                   'EBA.TEPC-ALL.D.H',
                   'EBA.TIDC-ALL.D.H',
                   'EBA.TPWR-ALL.D.H',
                   'EBA.WALC-ALL.D.H',
                   'EBA.WACM-ALL.D.H',
                   'EBA.WAUW-ALL.D.H']
    
    df={}
    for x in [[i] for i in demand_list]:
        BA = x[0]
        print(BA)
        d = EIAgov(tok, x)
        df[BA] = d.GetData()
        df[BA].index = pd.to_datetime(df[BA]['Date'])
        df[BA].drop(columns =['Date'], inplace=True)
        df[BA] = df[BA].resample('H').asfreq()
        
            
    df_all = pd.DataFrame(index = df['EBA.EPE-ALL.D.H'].index)    #EPE has least missing data
    for x in demand_list:
        df_all = pd.concat([df_all,df[x]], axis=1)
        


EBA.AVA-ALL.D.H
EBA.AZPS-ALL.D.H
EBA.BANC-ALL.D.H
EBA.BPAT-ALL.D.H
EBA.CHPD-ALL.D.H
EBA.CISO-ALL.D.H
EBA.DOPD-ALL.D.H
EBA.EPE-ALL.D.H
EBA.GCPD-ALL.D.H
EBA.IID-ALL.D.H
EBA.IPCO-ALL.D.H
EBA.LDWP-ALL.D.H
EBA.NEVP-ALL.D.H
EBA.NWMT-ALL.D.H
EBA.PACE-ALL.D.H
EBA.PACW-ALL.D.H
EBA.PGE-ALL.D.H
EBA.PSCO-ALL.D.H
EBA.PSEI-ALL.D.H
EBA.SRP-ALL.D.H
EBA.SCL-ALL.D.H
EBA.TEPC-ALL.D.H
EBA.TIDC-ALL.D.H
EBA.TPWR-ALL.D.H
EBA.WALC-ALL.D.H
EBA.WACM-ALL.D.H
EBA.WAUW-ALL.D.H


In [4]:
df_all.head()

,EBA.AVA-ALL.D.H,EBA.AZPS-ALL.D.H,EBA.BANC-ALL.D.H,EBA.BPAT-ALL.D.H,EBA.CHPD-ALL.D.H,EBA.CISO-ALL.D.H,EBA.DOPD-ALL.D.H,EBA.EPE-ALL.D.H,EBA.GCPD-ALL.D.H,EBA.IID-ALL.D.H,...,EBA.PSCO-ALL.D.H,EBA.PSEI-ALL.D.H,EBA.SRP-ALL.D.H,EBA.SCL-ALL.D.H,EBA.TEPC-ALL.D.H,EBA.TIDC-ALL.D.H,EBA.TPWR-ALL.D.H,EBA.WALC-ALL.D.H,EBA.WACM-ALL.D.H,EBA.WAUW-ALL.D.H
Date,,,,,,,,,,,,,,,,,,,,,
2015-07-01 07:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4875.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-07-01 08:00:00,1192.0,4478.0,2513.0,NaN,434.0,31486.0,177.0,925.0,663.0,505.0,...,4618.0,3066.0,4103.0,873.0,1605.0,408.0,468.0,1119.0,NaN,NaN
2015-07-01 09:00:00,1108.0,4227.0,2275.0,NaN,422.0,28989.0,169.0,856.0,654.0,482.0,...,4427.0,2865.0,3897.0,833.0,1537.0,380.0,441.0,1018.0,NaN,NaN
2015-07-01 10:00:00,1058.0,4016.0,2104.0,NaN,416.0,27416.0,161.0,839.0,646.0,474.0,...,4344.0,2735.0,3694.0,802.0,1487.0,357.0,420.0,1039.0,NaN,NaN
2015-07-01 11:00:00,1024.0,3879.0,1988.0,NaN,413.0,26388.0,160.0,827.0,642.0,450.0,...,4374.0,2710.0,3590.0,796.0,1470.0,342.0,417.0,1019.0,NaN,NaN


In [5]:
print(df_all.isnull().sum())


EBA.AVA-ALL.D.H      900
EBA.AZPS-ALL.D.H     652
EBA.BANC-ALL.D.H     744
EBA.BPAT-ALL.D.H    1952
EBA.CHPD-ALL.D.H     455
EBA.CISO-ALL.D.H     455
EBA.DOPD-ALL.D.H    2195
EBA.EPE-ALL.D.H      401
EBA.GCPD-ALL.D.H     432
EBA.IID-ALL.D.H      832
EBA.IPCO-ALL.D.H     534
EBA.LDWP-ALL.D.H    1019
EBA.NEVP-ALL.D.H    1314
EBA.NWMT-ALL.D.H     938
EBA.PACE-ALL.D.H    3144
EBA.PACW-ALL.D.H    4273
EBA.PGE-ALL.D.H      915
EBA.PSCO-ALL.D.H     820
EBA.PSEI-ALL.D.H     721
EBA.SRP-ALL.D.H     1474
EBA.SCL-ALL.D.H      535
EBA.TEPC-ALL.D.H    1687
EBA.TIDC-ALL.D.H     525
EBA.TPWR-ALL.D.H     506
EBA.WALC-ALL.D.H     673
EBA.WACM-ALL.D.H    1249
EBA.WAUW-ALL.D.H    4758
dtype: int64


In [6]:
df_all.tail()

,EBA.AVA-ALL.D.H,EBA.AZPS-ALL.D.H,EBA.BANC-ALL.D.H,EBA.BPAT-ALL.D.H,EBA.CHPD-ALL.D.H,EBA.CISO-ALL.D.H,EBA.DOPD-ALL.D.H,EBA.EPE-ALL.D.H,EBA.GCPD-ALL.D.H,EBA.IID-ALL.D.H,...,EBA.PSCO-ALL.D.H,EBA.PSEI-ALL.D.H,EBA.SRP-ALL.D.H,EBA.SCL-ALL.D.H,EBA.TEPC-ALL.D.H,EBA.TIDC-ALL.D.H,EBA.TPWR-ALL.D.H,EBA.WALC-ALL.D.H,EBA.WACM-ALL.D.H,EBA.WAUW-ALL.D.H
Date,,,,,,,,,,,,,,,,,,,,,
2018-08-23 18:00:00,1338.0,4532.0,2074.0,NaN,190.0,31064.0,NaN,NaN,671.0,NaN,...,5594.0,3485.0,4758.0,1059.0,NaN,356.0,526.0,1144.0,3323.0,107.0
2018-08-23 19:00:00,1387.0,4759.0,2150.0,NaN,195.0,31688.0,NaN,NaN,685.0,NaN,...,5869.0,3521.0,5097.0,1065.0,NaN,366.0,525.0,1194.0,3412.0,110.0
2018-08-23 20:00:00,1434.0,5037.0,2248.0,NaN,203.0,32516.0,NaN,NaN,696.0,NaN,...,6102.0,3539.0,5408.0,1068.0,NaN,382.0,522.0,1248.0,3442.0,114.0
2018-08-23 21:00:00,1482.0,5336.0,2369.0,NaN,201.0,33728.0,NaN,NaN,711.0,NaN,...,6393.0,3570.0,5672.0,1057.0,NaN,400.0,540.0,1263.0,3453.0,115.0
2018-08-23 22:00:00,1516.0,5539.0,2488.0,NaN,206.0,35172.0,NaN,NaN,719.0,NaN,...,6634.0,3590.0,5811.0,1054.0,NaN,422.0,128.0,1284.0,3551.0,119.0


In [8]:
len(df_all.columns)

27

In [9]:
df_all.describe()

,EBA.AVA-ALL.D.H,EBA.AZPS-ALL.D.H,EBA.BANC-ALL.D.H,EBA.BPAT-ALL.D.H,EBA.CHPD-ALL.D.H,EBA.CISO-ALL.D.H,EBA.DOPD-ALL.D.H,EBA.EPE-ALL.D.H,EBA.GCPD-ALL.D.H,EBA.IID-ALL.D.H,...,EBA.PSCO-ALL.D.H,EBA.PSEI-ALL.D.H,EBA.SRP-ALL.D.H,EBA.SCL-ALL.D.H,EBA.TEPC-ALL.D.H,EBA.TIDC-ALL.D.H,EBA.TPWR-ALL.D.H,EBA.WALC-ALL.D.H,EBA.WACM-ALL.D.H,EBA.WAUW-ALL.D.H
count,26692.000000,2.694000e+04,26848.000000,25640.000000,27137.000000,27137.000000,25397.000000,27191.000000,27160.000000,26760.000000,...,26772.000000,26871.000000,26118.000000,27057.000000,25905.000000,27067.000000,27086.000000,26919.000000,26343.000000,22834.000000
mean,1356.463127,3.742700e+03,2035.997206,6297.846880,237.620739,26587.435347,176.791747,977.126770,567.948012,416.737556,...,4991.986740,3406.589744,3424.255877,1110.101489,1684.020459,306.795877,555.229270,1154.497158,2942.027066,92.834019
std,466.437654,1.447026e+04,537.410696,990.477522,102.706333,5176.542590,44.683980,283.800133,81.997369,183.090255,...,928.266512,640.678941,1848.013905,218.061683,689.682195,87.267497,112.254126,3983.630916,380.938506,19.096696
min,748.000000,2.023000e+03,0.000000,2600.000000,-85.000000,18068.000000,0.000000,-7342.000000,362.000000,-29.000000,...,0.000000,679.000000,-755.000000,0.000000,890.000000,0.000000,-12.000000,-92392.000000,-933.000000,0.000000
25%,1148.000000,2.805000e+03,1696.000000,5620.000000,159.000000,22836.000000,143.000000,787.000000,507.000000,280.750000,...,4381.000000,2958.000000,2643.000000,974.000000,1399.000000,250.000000,480.000000,885.000000,2691.000000,79.000000
50%,1327.000000,3.275000e+03,1905.000000,6214.000000,201.000000,25617.000000,167.000000,901.000000,558.000000,357.000000,...,4890.000000,3379.000000,3041.500000,1109.000000,1563.000000,284.000000,540.000000,1016.000000,2889.000000,90.000000
75%,1558.000000,4.154000e+03,2151.000000,6886.000000,294.000000,28762.000000,205.000000,1108.000000,619.000000,515.000000,...,5422.000000,3823.000000,3934.000000,1236.000000,1843.000000,337.000000,625.000000,1184.000000,3162.000000,105.000000
max,58854.000000,1.680538e+06,4763.000000,11827.000000,591.000000,49899.000000,397.000000,5150.000000,982.000000,1081.000000,...,60576.000000,5504.000000,247000.000000,11583.000000,66155.000000,653.000000,998.000000,401364.000000,14528.000000,168.000000


In [11]:
min(df_all.index)

Timestamp('2015-07-01 07:00:00', freq='H')

In [12]:
max(df_all.index)

Timestamp('2018-08-23 22:00:00', freq='H')